### Dimensions in CNNs

* Understanding dimensions will help you make accurate tradeoffs between model size and performance. As you'll see, some parameters have a much bigger impact on model size than others.

**Question**

* H = height, W = width, D = depth
* We have an input of shape 32x32x3 (HxWxD)
* 20 filters of shape 8x8x3 (HxWxD)
* A stride of 2 for both the height and width (S)
* With padding of size 1 (P)
* Recall the formula for calculating the new height or width:

> <b style='color:blue'>new_height</b> = <b style='color:red'>(input_height - filter_height + 2 * P)/S + 1</b>

> <b style='color:blue'>new_width</b> = <b style='color:red'>(input_width - filter_width + 2 * P)/S + 1</b>

* What's the shape of the output? 

We can get the new height and width with the formula resulting in:

> (32 - 8 + 2 * 1)/2 + 1 = 14 

> (32 - 8 + 2 * 1)/2 + 1 = 14

The new depth is equal to the number of filters, which is 20.
This would correspond to the following code:

```python
input = tf.placeholder(tf.float32, (None, 32, 32, 3))

# filter has the dimension: (height, width, input_depth, output_depth)
filter_weights = tf.Variable(tf.truncated_normal((8, 8, 3, 20))) 

filter_bias = tf.Variable(tf.zeros(20))

# stride has the dimension: (batch, height, width, depth)
strides = [1, 2, 2, 1] 
padding = 'SAME'
conv = tf.nn.conv2d(input, filter_weights, strides, padding) + filter_bias
```

Note the output shape of conv will be [1, 16, 16, 20]. It's 4D to account for batch size, but more importantly, it's not [1, 14, 14, 20]. 

> This is because the padding algorithm TensorFlow uses is not exactly the same as the one above. 

An alternative algorithm is to switch padding from 'SAME' to 'VALID' which would result in an output shape of [1, 13, 13, 20]. If you're curious how padding works in TensorFlow, read this [document](https://www.tensorflow.org/api_guides/python/nn#Convolution).

The filter is applied to image patches of the same size as the filter and strided according to the strides argument, which has the dimension: (batch, height, width, depth). 

> strides = [1, 1, 1, 1] applies the filter to a patch at every offset.

> strides = [1, 2, 2, 1] applies the filter to every other image patch in each dimension.

In summary TensorFlow uses the following equation for 'SAME' vs 'VALID'

<b>SAME Padding</b>, the output height and width are computed as:

> <b style='color:blue'>out_height</b> = <b style='color:red'>ceil( float(in_height) / float(strides[1]) )</b>

> <b style='color:blue'>out_width</b> = <b style='color:red'>ceil( float(in_width) / float(strides[2]) )</b>

<b>VALID Padding</b>, the output height and width are computed as:

> <b style='color:blue'>out_height</b> = <b style='color:red'>ceil( float(in_height - filter_height + 1 ) / float(strides[1]))</b>

> <b style='color:blue'>out_width</b> = <b style='color:red'>ceil( float(in_width - filter_width + 1 ) / float(strides[2]))</b>